In [1]:
#Create a Spark Session
from pyspark.sql import SparkSession
import findspark

findspark.init()

#Note that we have set parallelism to 8
spark = SparkSession\
            .builder\
            .appName("LearnerExerciseJob")\
            .config("spark.sql.shuffle.partitions", 8)\
            .config("spark.default.parallelism", 8)\
            .config("spark.sql.warehouse.dir", "spark-warehouse") \
            .enableHiveSupport() \
            .master("local[2]")\
            .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/13 09:30:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/13 09:30:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### 06.02 Data Loading

#### Read CSV file

In [2]:
#Load the student score file
raw_student_data = spark\
                .read\
                .option("inferSchema", "true")\
                .option("header", "true")\
                .csv("datasets/student_scores.csv")

#Display schema and data
raw_student_data.printSchema()
raw_student_data.show(5)

root
 |-- Student: string (nullable = true)
 |-- Subject: string (nullable = true)
 |-- ClassScore: double (nullable = true)
 |-- TestScore: double (nullable = true)

+-------+---------+----------+---------+
|Student|  Subject|ClassScore|TestScore|
+-------+---------+----------+---------+
|   Katy|     Math|      0.95|     2.37|
|   Katy|Chemistry|       0.5|     1.18|
|   Katy|  Physics|      0.48|     1.37|
|   Katy|  Biology|      0.75|     2.79|
|   Mike|     Math|      0.45|     1.79|
+-------+---------+----------+---------+
only showing top 5 rows



#### Create partitioned HDFS Store

In [3]:
#Store as parquet file for better performance
raw_student_data.write\
            .option("compression", "gzip")\
            .partitionBy("Subject")\
            .parquet(path="dummy_hdfs/partitioned_student",
                    mode="overwrite");

#### Read from partitioned store to a dataframe

In [4]:
student_data = spark\
                .read\
                .parquet("dummy_hdfs/partitioned_student")

print("Partitions in student data :",student_data.rdd.getNumPartitions())

Partitions in student data : 4


### 06.03 Total Score Analytics

#### Compute total score by student and subject

In [5]:
from pyspark.sql.functions import col

#Create new column TotalScore by adding ClassScore and TestScore
total_score_df = student_data.withColumn("TotalScore",
                                         col("ClassScore") + col("TestScore"))
total_score_df.show(5)

+-------+----------+---------+-------+------------------+
|Student|ClassScore|TestScore|Subject|        TotalScore|
+-------+----------+---------+-------+------------------+
|   Katy|      0.95|     2.37|   Math|3.3200000000000003|
|   Mike|      0.45|     1.79|   Math|              2.24|
|    Bob|      0.36|     2.37|   Math|              2.73|
|   Lisa|      0.33|     2.86|   Math|              3.19|
|   John|      0.27|      1.2|   Math|              1.47|
+-------+----------+---------+-------+------------------+
only showing top 5 rows



#### Print total score for physics for all students

In [6]:
physics_score= total_score_df.where(col("Subject") == 'Physics')
physics_score.show()

#show the execution plan
print("\n--------------------------EXPLAIN--------------------------")
physics_score.explain()
print("-------------------------END EXPLAIN-----------------------\n")

+-------+----------+---------+-------+------------------+
|Student|ClassScore|TestScore|Subject|        TotalScore|
+-------+----------+---------+-------+------------------+
|   Katy|      0.48|     1.37|Physics|              1.85|
|   Mike|      0.34|     2.72|Physics|              3.06|
|    Bob|      0.93|     2.89|Physics|3.8200000000000003|
|   Lisa|      0.42|     2.34|Physics|              2.76|
|   John|      0.82|      2.8|Physics|3.6199999999999997|
+-------+----------+---------+-------+------------------+


--------------------------EXPLAIN--------------------------
== Physical Plan ==
*(1) Project [Student#56, ClassScore#57, TestScore#58, Subject#59, (ClassScore#57 + TestScore#58) AS TotalScore#64]
+- *(1) ColumnarToRow
   +- FileScan parquet [Student#56,ClassScore#57,TestScore#58,Subject#59] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/Users/linkedin/ExerciseFiles/dummy_hdfs/partitioned_student], PartitionFilters: [isnotnull(S

### 06.04 Compute average total score for each student across subjects

In [7]:
#cache the total score dataframe
total_score_df.persist()

avg_score_df = total_score_df\
                .groupBy("Student")\
                .avg("TotalScore")

avg_score_df.show()

#show the execution plan
print("\n--------------------------EXPLAIN--------------------------")
avg_score_df.explain()
print("-------------------------END EXPLAIN-----------------------\n")

+-------+------------------+
|Student|   avg(TotalScore)|
+-------+------------------+
|   Katy|            2.5975|
|   Mike|             2.455|
|   Lisa|2.3899999999999997|
|    Bob|3.0150000000000006|
|   John|            2.8525|
+-------+------------------+


--------------------------EXPLAIN--------------------------
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Student#56], functions=[avg(TotalScore#64)])
   +- Exchange hashpartitioning(Student#56, 8), ENSURE_REQUIREMENTS, [plan_id=188]
      +- HashAggregate(keys=[Student#56], functions=[partial_avg(TotalScore#64)])
         +- InMemoryTableScan [Student#56, TotalScore#64]
               +- InMemoryRelation [Student#56, ClassScore#57, TestScore#58, Subject#59, TotalScore#64], StorageLevel(disk, memory, deserialized, 1 replicas)
                     +- *(1) Project [Student#56, ClassScore#57, TestScore#58, Subject#59, (ClassScore#57 + TestScore#58) AS TotalScore#64]
                        +- *(1) 

### 06.05 Find top Student by Subject

In [8]:
#Find top score by subject
top_score_df = total_score_df\
                .groupBy("Subject")\
                .max("TotalScore")\
                .withColumnRenamed("max(TotalScore)","TopScore")

top_score_df.show()

#Find the student who had the top score
top_student_df = total_score_df.alias("a")\
                    .join(top_score_df.alias("b"),
                            (col("b.TopScore") == col("a.TotalScore")) & 
                              (col("b.Subject") == col("a.Subject"))) \
                    .select("a.Subject", "a.Student", "b.TopScore")

top_student_df.show()
                          

+---------+------------------+
|  Subject|          TopScore|
+---------+------------------+
|     Math|3.3200000000000003|
|  Physics|3.8200000000000003|
|Chemistry|3.1999999999999997|
|  Biology|              3.54|
+---------+------------------+

+---------+-------+------------------+
|  Subject|Student|          TopScore|
+---------+-------+------------------+
|     Math|   Katy|3.3200000000000003|
|  Physics|    Bob|3.8200000000000003|
|Chemistry|   John|3.1999999999999997|
|  Biology|   Katy|              3.54|
+---------+-------+------------------+

